### Priprava podatkov

In [ ]:
import pandas as pd

def to_ura(series):
    ure = []
    for i, val in series.items():
        ure.append(int(val.split(":")[0]))
    return pd.Series(ure, dtype="int")

def to_mesec(series):
    mesec = []
    for i, val in series.items():   
        mesec.append(int(str("{:.3f}".format(val))[:-5])) 
    return pd.Series(mesec, dtype="int")

df = None
for year in range(2012, 2022):
    df_temp = pd.read_csv("data/data" + str(year) + ".csv", delimiter=";", encoding="utf8", low_memory=False)
    df_temp.iloc[:, 0] = df_temp.iloc[:, 0] + year * 1000_000
    df_temp.iloc[:, 1] = to_mesec(df_temp.iloc[:, 1])
    df_temp.iloc[:, 2] = to_ura(df_temp.iloc[:, 2])
    df_temp.iloc[:, 3] = df_temp.iloc[:, 3].replace(["PONEDELJEK", "TOREK", "SREDA", "ČETRTEK", "PETEK", "SOBOTA", "NEDELJA"], [1, 2, 3, 4, 5, 6, 7]).astype(int)
    df_temp.iloc[:, 5] = df_temp.iloc[:, 5].replace(["DA", "NE"], [1, 0]).astype("bool")
    df_temp.iloc[:, 8] = df_temp.iloc[:, 8].replace(["GOSPODARSKA", "SPLOŠNA"], [1, 0]).astype("bool")
    df_temp.iloc[:, 9] = df_temp.iloc[:, 9].replace(["ORGANIZIRANA", " "], [1, 0]).astype("bool")
    df_temp.iloc[:, 10] = df_temp.iloc[:, 10].replace(["MLADOLETNIŠKA", " "], [1, 0]).astype("bool")
    df_temp.iloc[:, 11] = df_temp.iloc[:, 11].replace(["DA", "NE"], [1, 0]).astype("bool")
    df_temp.iloc[:, 31] = df_temp.iloc[:, 31].replace(["DA", "NE"], [1, 0]).astype("bool")
    df_temp.iloc[:, 31] = df_temp.iloc[:, 31].replace(["BREZ"], [0]).astype("bool")

    df = pd.concat([df, df_temp], ignore_index = True)

### 1. Ali sej je karantena spremenila pogostost družinskega nasilja?

### 2. Ali se je med karanteno spremenila pogostost drugih kaznivih dejanj?

### 3. Kako je uvedba policijske ure vplivala na kriminal v nočnem času?